In [ ]:
import numpy as np
import pandas as pd
import math
import sklearn
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation

In [ ]:
dataset = pd.read_csv("/content/fer2013.csv")

In [ ]:
dataset = dataset[dataset['Usage'] == "Training"]
dataset = dataset[['pixels', 'emotion']]

In [ ]:
image_dataset = dataset.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))

In [ ]:
image_dataset = np.stack(image_dataset, axis=0)

In [ ]:
le = LabelEncoder()
img_labels = le.fit_transform(dataset.emotion)
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

(28709, 7)

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(image_dataset, img_labels, shuffle=True, stratify=img_labels, test_size=0.1, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((25838, 48, 48, 1), (2871, 48, 48, 1), (25838, 7), (2871, 7))

In [ ]:
X_train = X_train / 255.
X_valid = X_valid / 255.

In [ ]:
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

In [ ]:
model = tf.keras.models.Sequential(name='DCNN')

model.add(Conv2D(filters=64,kernel_size=(5,5),input_shape=(img_width, img_height, img_depth),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_1'))
model.add(BatchNormalization(name='batchnorm_1'))
model.add(Conv2D(filters=64,kernel_size=(5,5),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_2'))
model.add(BatchNormalization(name='batchnorm_2'))

model.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
model.add(Dropout(0.4, name='dropout_1'))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_3'))
model.add(BatchNormalization(name='batchnorm_3'))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_4'))
model.add(BatchNormalization(name='batchnorm_4'))

model.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
model.add(Dropout(0.4, name='dropout_2'))

model.add(Conv2D(filters=256,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_5'))
model.add(BatchNormalization(name='batchnorm_5'))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',name='conv2d_6'))
model.add(BatchNormalization(name='batchnorm_6'))

model.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_3'))
model.add(Dropout(0.5, name='dropout_3'))

model.add(Flatten(name='flatten'))

model.add(Dense(128,activation='elu',kernel_initializer='he_normal',name='dense_1'))
model.add(BatchNormalization(name='batchnorm_7'))

model.add(Dropout(0.6, name='dropout_4'))

model.add(Dense(num_classes,activation='softmax',name='out_layer'))

model.compile(
loss='categorical_crossentropy',
optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam'),
metrics=['accuracy']
)

model.summary()

Model: "DCNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        1664      
                                                                 
 batchnorm_1 (BatchNormaliza  (None, 48, 48, 64)       256       
 tion)                                                           
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 64)        102464    
                                                                 
 batchnorm_2 (BatchNormaliza  (None, 48, 48, 64)       256       
 tion)                                                           
                                                                 
 maxpool2d_1 (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 dropout_1 (Dropout)         (None, 24, 24, 64)        0      

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=0.00005,patience=11,verbose=1,restore_best_weights=True,)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',factor=0.5,patience=7,min_lr=1e-7,verbose=1,)

In [ ]:
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(zoom_range=0.15, horizontal_flip=True)
train_image_generator.fit(X_train)

In [ ]:
generated_images = train_image_generator.flow(X_train, y_train, batch_size=32)

In [ ]:
hist = model.fit(
    generated_images,
    validation_data=(X_valid, y_valid),
    steps_per_epoch=len(X_train) / 32,
    epochs=100,
    callbacks=[early_stopping,lr_scheduler],
)

Epoch 1/100
807/807 [==============================] - 31s 38ms/step - loss: 0.7080 - accuracy: 0.7457 - val_loss: 0.9536 - val_accuracy: 0.6695 - lr: 5.0000e-04
Epoch 2/100
807/807 [==============================] - 29s 36ms/step - loss: 0.7066 - accuracy: 0.7386 - val_loss: 0.9650 - val_accuracy: 0.6667 - lr: 5.0000e-04
Epoch 3/100
807/807 [==============================] - 27s 34ms/step - loss: 0.7042 - accuracy: 0.7439 - val_loss: 0.9555 - val_accuracy: 0.6677 - lr: 5.0000e-04
Epoch 4/100
807/807 [==============================] - 28s 35ms/step - loss: 0.6953 - accuracy: 0.7460 - val_loss: 0.9629 - val_accuracy: 0.6614 - lr: 5.0000e-04
Epoch 5/100
807/807 [==============================] - 27s 34ms/step - loss: 0.6932 - accuracy: 0.7474 - val_loss: 0.9639 - val_accuracy: 0.6674 - lr: 5.0000e-04
Epoch 6/100
807/807 [==============================] - 28s 35ms/step - loss: 0.6890 - accuracy: 0.7480 - val_loss: 0.9749 - val_accuracy: 0.6614 - lr: 5.0000e-04
Epoch 7/100
807/807 [=======

In [ ]:
model.save('/content/PredictorWithHundredEpochs.h5')